In [1]:
from keybert import KeyBERT
from transformers import BertModel
from konlpy.tag import Kkma
from konlpy.tag import Okt
from kiwipiepy import Kiwi
from collections import Counter

In [2]:
model = BertModel.from_pretrained('skt/kobert-base-v1')
kw_model = KeyBERT(model)

In [3]:
class KewordExtractor:
    def __init__(self):
        self.kiwi = Kiwi()
        self.okt = Okt()
        self.kkma = Kkma()
        
        self.df = None
        self.nouns = []
        
    def setDataFrame(self, dataFrame):
        self.df = dataFrame
        
    def normText(self, text):
        return self.okt.normalize(text)
    
    def getNouns(self, textLine):
        #라인 하나씩 받아서 NOUNS 리턴
        #kiwi NNP
        text = self.normText(textLine)
        
        nouns = []
        nouns = nouns + self.kiwiMorph(text)
        nouns = nouns + self.oktMorph(text)
        nouns = nouns + self.kkmaMorph(text)
        
        result = []
        counter = Counter(nouns)
        for k, v in dict(counter).items():
            if v>2 :
                result.append(k)
                
        return result
        
        
        
        
    def kkmaMorph(self, input_data) : #형태소 분석
        preprocessed = self.kkma.pos(input_data)
        ret = []
        for data in preprocessed:
            if(data[1]=="NNG"):
                ret.append(data[0])
        return ret
    def oktMorph(self, input_data):
        return self.okt.nouns(input_data)
    
    def kiwiMorph(self, input_data):
        results = []
        result = self.kiwi.analyze(input_data)
        for token, pos, _, _ in result[0][0]:
            if len(token) != 1 and pos=='NNG':
                results.append(token)
        return results